# Final Project - Korean to English Translation

- Sequence to Sequence 모델의 대표적인 한국어-영어 번역을 Encoder-decoder, Attention, Convolution, 그리고 Transformers 기반으로 구현
- 수업시간에 살펴본 Pytorch Seq to Seq 모델 (https://github.com/bentrevett/pytorch-seq2seq)을 참조로 하여 한국어와 영어 형태소분석되고 의존관계로 되어 있는 파일을 프로세싱하여 두 언어의 parallel 데이터 쌍으로 만들고 이를 학습하여 모델별로 Perplexity가 어떻게 달라지는지 살펴 보고, 가장 성능이 좋은 모델을 근간으로 해서 Inference로 한국어 문장을 입력하면 대응되는 영어 번역이 출력될 수 있도록 구현
- 반드시 다음 세 모델에 대해서 PPL와 BLEU score가  다 체크되어야 함. (Packed) Encoder-Decoder, Convolutional Seq to Seq, Transformers ...
- 세 모델 중에 학습이 제대로 이루어지지 않는 경우, PPL이나 BLEU가 문제가 있는 경우 이를 Fix하려고 시도해 보라.
- Inference시에 unk인 단어를 로마자화해서 번역에 나타날 수 있도록 시도해 볼 것(참고할 수 있는 사이트 중 하나 https://github.com/osori/korean-romanizer)

- 개인적으로 하거나 최대 두명까지 그룹 허용. 
- 이 노트북 화일에 이름을 변경하여 작업하고 제출. 제출시 화일명을 Assignment4_[DS또는 CL]_학과_이름.ipynb

## Data 1
- 첨부된 ko-en-en.parse.syn은 330,974 한국어 문장에 대응되는 영어문장이 품사와 구문분석이 되어 있는 파일이고 ko-en-ko.parse.syn은 이에 대응되는 한국어 문장이 형태소와 구문분석이 되어 있는 파일이다.

(ROOT (S (NP (NNP Flight) (NNP 007)) (VP (MD will) (VP (VB stay) (PP (IN on) (NP (NP (DT the) (NN ground)) (PP (IN for) (NP (CD one) (NN hour))))))) (. .)))


<id 1>
<sent 1>
1       2       NP      777/SN
2       6       NP_SBJ  항공편/NNG|은/JX
3       4       NP      1/SN|시간/NNG
4       6       NP_AJT  동안/NNG
5       6       NP_AJT  지상/NNG|에/JKB
6       7       VP      머물/VV|게/EC
7       0       VP      되/VV|ㅂ니다/EF|./SF
</sent>
</id>

    - 이 두 파일을 프로세싱하여 한-영 병렬 데이터로 만들고 이를 학습 및 테스트 데이터로 사용한다.
    - Hint: 구조화된 데이터를 프로세싱하기 위해서는 nltk의 모듈을 사용할 수 있다.

    - 한국어 형태소 분석된 단위를 어절별로 결합할 수 있고, 분석된 채로 그대로 사용할 수도 있다.
    - 두 언어의 어순을 비슷하게 데이터를 만들어 학습할 수도 있고, 번역의 성능을 높이기 위해 다양한 형태로 재구조화 할 수 있다.

## Data 2
[Korean Parallel Corpora](https://github.com/jungyeul/korean-parallel-corpora)에는 Korean-English-jhe, Korean-English-news-v1의 병렬 데이터가 있다. 
- 이 데이터를 다운 받아 Data1의 자료와 합쳐서 사용할 수 있다
- 이 경우 형태소 분석이 된 경우와 그렇지 않은 자료가 있으니, Data1의 자료와 합칠 때 형태소 분석을 하거나 아니면 어절 단위로 결합하여 할 수도 있다.
- 전체적인 일관성 및 inference를 위해서 형태소 분석된 것이 더 좋을 수 있는데, 이 경우 형태소 분석되지 않는 데이터는 새로 형태소 분석을 할 필요. 단 형태소 분석 단위가 일치하는지 알아볼 필요. 형태소 분석 단위가 완전히 일치하지 않는다면 가급적 분석단위 일치도가 높은 형태소 분석기를 선택할 필요

## 구현,실험 전체적인 설명 및 분석 

## Your Code

In [89]:
from tqdm import tqdm
from nltk import Tree

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.legacy import data
from torchtext.legacy import datasets

import numpy as np
from einops import rearrange, reduce

import json # save parsed list

In [90]:
with open('ko-en.ko.parse','r') as f:
    ko = list(f)

## split 으로 구분할 수 있는 형태로 전처리
# 문장 단위로 분리하기
# 문장 내부에서 token 추출하기

def extract_sentence(token_list):
    '''extract <sent #> sth </sent>'''
    sentence_list = []
    sentence = ''
    in_sentence = False

    for tok in token_list:
        if '<sent' in tok:
            in_sentence = True

        if in_sentence:
            sentence += tok

        if '</sent>' in tok:
            sentence_list.append(sentence)
            in_sentence = False
            sentence = ''
            
    return sentence_list

def extract_token(segment_with_morpheme) -> list:
    '''split token/morpheme|...|token/morpheme'''
    token_list = []
    if '|' in segment_with_morpheme: # multiple token in a sement
        for token in segment_with_morpheme.split(sep = '|'):
            token_list.append(token.split(sep = '/')[0])
        
    else: # multiple token in a sement
        token_list.append(segment_with_morpheme.split(sep = '/')[0])
        
    return token_list

def preprocess(ko):
    processed_sentence_list = []
    sentence_list = extract_sentence(ko)
    
    for sentence in tqdm(sentence_list):
        processed_token_list = []
        for segment in sentence.split(sep = '\n'):
            if '\t' in segment:
                token_with_morpheme = segment.split(sep = '\t')[-1]
                processed_token_list += extract_token(token_with_morpheme)
                
        processed_token_list.insert(0, '<sos>')
        processed_token_list.append('<eos>')
        processed_sentence_list.append(processed_token_list)

    return processed_sentence_list

processed_ko = preprocess(ko)

100%|██████████| 349874/349874 [00:06<00:00, 57610.25it/s]


In [91]:
# English data
with open('ko-en.en.parse.syn') as f:
    en = f.readlines()

processed_en = []
for idx, _ in tqdm(enumerate(en)):
    t = Tree.fromstring(en[idx])
    processed_token_list = t.leaves()
    
    processed_token_list.insert(0, '<sos>')
    processed_token_list.append('<eos>')
    
    processed_en.append(processed_token_list)

330974it [00:27, 12193.65it/s]


In [92]:
# save parsed dataset
import json

ko_en_json = [{'ko' : ko, 'en' : en} for ko, en in zip(processed_ko[:32*2], processed_en[:32*2])]
with open('ko_en_parsed.json', 'w') as fp:
    json.dump(ko_en_json, fp)
    
# check whether properly saved or not
with open('ko_en_parsed.json', 'r') as fp:
    ko_en = json.loads(next(fp)) #!#
    
# print(ko_en)

In [93]:
## torchtext 에 전달하기
# json file 로 만들기
ko_en_json = [{'ko' : ko, 'en' : en} for ko, en in zip(processed_ko[:32*2], processed_en[:32*2])]
with open('ko_en_parsed.json', 'w') as fp:
    json.dump(ko_en_json, fp)

KOREAN = data.Field( 
    init_token = '<sos>', 
    eos_token = '<eos>', 
    lower = True, 
    include_lengths = True
)

ENGLISH = data.Field( 
    init_token = '<sos>', 
    eos_token = '<eos>', 
    lower = True
)

fields  = {'ko' : ('ko', KOREAN), 'en' : ('en', ENGLISH)}

train_data, test_data = data.TabularDataset.splits(
                            path  = '/home/enkeejunior1/study_NLP/assignment/Assignment4/', #!#
                            train = 'ko_en_parsed.json',
                            test  = 'ko_en_parsed.json',
                            format = 'json',
                            fields = fields
)

KOREAN.build_vocab(train_data, min_freq = 2)
ENGLISH.build_vocab(train_data, min_freq = 2)

TypeError: descriptor 'lower' requires a 'str' object but received a 'list'

In [83]:
print(vars(train_data.examples[0])['ko'][0])

['<sos>', '777', '항공편', '은', '1', '시간', '동안', '지상', '에', '머물', '게', '되', 'ㅂ니다', '.', '<eos>']


In [84]:
BATCH_SIZE = 16

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, test_iterator = BucketIterator.splits(
    (train_data, test_data), 
     batch_size = BATCH_SIZE,
     sort_within_batch = True,
     sort_key = lambda x : len(x.ko),
     device = device)

NameError: name 'torch' is not defined

In [66]:


# word2vec 으로 embedding 시켜주기

In [ ]:
## model building
# preparation
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [47]:
## model 1 : (Packed) Encoder-Decoder

## model 2 : Convolutional Seq to Seq

## model 3 : Transformers

In [1]:
# Data2
from Korpora import Korpora

Korpora.fetch("korean_parallel_koen_news")
corpus = Korpora.load("korean_parallel_koen_news")

In [20]:
len(en)

330974

In [27]:
with open('ko-en.ko.parse','r') as f:
    ko = list(f)

In [28]:
len(ko)

3108134

In [83]:
ko[6000:7000]

['1\t2\tNP_OBJ\t영어/NNP\n',
 '2\t3\tVP\t공부/NNG|하/XSV|러/EC\n',
 '3\t0\tVP\t오/VV|았/EP|습니다/EF|./SF\n',
 '</sent>\n',
 '</id>\n',
 '\n',
 '<id 638>\n',
 '<sent 1>\n',
 '1\t2\tNP_OBJ\t영어/NNP\n',
 '2\t3\tVP\t공부/NNG|하/XSV|러/EC\n',
 '3\t0\tVP\t오/VV|았/EP|습니다/EF|./SF\n',
 '</sent>\n',
 '</id>\n',
 '\n',
 '<id 639>\n',
 '<sent 1>\n',
 '1\t2\tNP_OBJ\t영어/NNP\n',
 '2\t0\tVP\t공부/NNG|하/XSV|려고요/EF|./SF\n',
 '</sent>\n',
 '</id>\n',
 '\n',
 '<id 640>\n',
 '<sent 1>\n',
 '1\t2\tNP\t영어/NNP\n',
 '2\t3\tVP\t연수/NNG|하/XSV|러/EC\n',
 '3\t0\tVP\t오/VV|았/EP|습니다/EF|./SF\n',
 '</sent>\n',
 '</id>\n',
 '\n',
 '<id 641>\n',
 '<sent 1>\n',
 '1\t2\tNP_OBJ\t영어/NNP\n',
 '2\t0\tVP\t잘/MAG|하/XSV|시/EP|어요/EF|?/SF\n',
 '</sent>\n',
 '</id>\n',
 '\n',
 '<id 642>\n',
 '<sent 1>\n',
 '1\t2\tNP_SBJ\t영어/NNP|가/JKS\n',
 '2\t4\tVP\t서투르/VA|어/EC\n',
 '3\t4\tNP_AJT\t영어/NNP|로/JKB\n',
 '4\t5\tVP_MOD\t설명/NNG|하/XSV|ㄹ/ETM\n',
 '5\t6\tNP_SBJ\t수/NNB\n',
 '6\t0\tVP\t없/VA|습니다/EF|./SF\n',
 '</sent>\n',
 '</id>\n',
 '\n',
 '<id 643>\n',
 '<sent 1>\n'

In [7]:
ko[2]

'1\t2\tNP\t777/SN\n'

In [11]:


t = Tree.fromstring(en[0])
t = Tree.fromstring(ko[0])

#print t.leaves()
print (' '.join(t.leaves()))

ValueError: Tree.read(): expected '(' but got '<id'
            at index 0.
                "<id 1> "
                 ^

3108134

In [ ]:
## File Processing

## Bleu Score

## Inference

In [ ]:
## 형태소 분석을 할 경우

sen_list = [
'모든 액체 , 젤 , 에어로졸 등 은 1 커트 짜리 여닫이 투명 봉지 하나 에 넣 어야 하 ㅂ니다 .',
'미안 하 지만 , 뒷쪽 아이 들 의 떠들 는 소리 가 커 어서 , 광화문 으로 가 아고 싶 은데 표 를 바꾸 어 주 시 겠 어요 ?',
'은행 이 너무 멀 어서 안 되 겠 네요 . 현찰 이 필요 하면 돈 을 훔치 시 어요',
'아무래도 분실 하 ㄴ 것 같 으니 분실 신고서 를 작성 하 아야 하 겠 습니다 . 사무실 로 같이 가 시 ㄹ 까요 ?',
'부산 에서 코로나 확진자 가 급증 하 아서 병상 이 부족하 아 지자  확진자 20명 을 대구 로 이송하 ㄴ다 .',
'변기 가 막히 었 습니다 .',
'그 바지 좀 보이 어 주 시 ㅂ시오 . 이거 얼마 에 사 ㄹ 수 있 는 것 이 ㅂ니까 ?',
'비 가 오 아서 백화점 으로 가지 말 고 두타 로 가 았 으면 좋 겠 습니다 .',
'속 이 안 좋 을 때 는 죽 이나 미음 으로 아침 을 대신 하 ㅂ니다',
'문 대통령 은 집단 이익 에서 벗어 나 아 라고 말 하 었 다 .',
'이것 좀 먹어 보 ㄹ 몇 일 간 의 시간 을 주 시 어요 .',
'이날 개미군단 은 외인 의 물량 을 모두 받 아 내 었 다 .',
'통합 우승 의 목표 를 달성하 ㄴ NC 다이노스 나성범 이 메이저리그 진출 이라는 또 다른 꿈 을 향하 어 나아가 ㄴ다 .',
'이번 구조 조정 이 제품 을 효과 적 으로 개발 하 고 판매 하 기 위하 ㄴ 회사 의 능력 강화 조처 이 ㅁ 을 이해 하 아 주 시 리라 생각 하 ㅂ니다 .',
'요즘 이 프로그램 녹화 하 며 많은 걸 느끼 ㄴ다 '
'배낭 여행 은 우리 들 이 어리었을 때 허용 되 지 않 으면 우울 하 아 하 ㄴ다'
'그 소녀 는 단지 늑대 처럼 울부짖 을 수 있 을 뿐 이 었 다']


In [ ]:
## 어절단위로 결합하여 할 경우

sen_list = [
'모든 액체, 젤, 에어로졸 등은 1커트 짜리 여닫이 투명 봉지 하나에 넣어야 합니다.',
'미안하지만 , 뒷쪽 아이들의 떠드는 소리가 커서 , 광화문으로 가고 싶은데 표를 바꾸어 주시겠어요?',
'은행이 너무 멀어서 안되겠네요. 현찰이 필요하면 돈을 훔치세요',
'아무래도 분실한 것 같으니 분실신고서를 작성해야 하겠습니다. 사무실로 같이 가실까요?',
'부산에서 코로나 확진자가 급증해서 병상이 부족해지자  확진자 20명을 대구로 이송한다.',
'변기가 막히었습니다 .',
'그 바지 좀 보여주십시오. 이거 얼마에 살 수 있는 것입니까?',
'비가 와서 백화점으로 가지 말고 두타로 갔으면 좋겠습니다 .',
'속이 안좋을 때는 죽이나 미음으로 아침을 대신 합니다',
'문 대통령은 집단이익에서 벗어나라고 말하였다.',
'이것 좀 먹어 볼 몇 일 간의 시간을 주세요.',
'이날 개미군단은 외인의 물량을 모두 받아내었다.',
'통합 우승의 목표를 달성한 NC 다이노스 나성범이 메이저리그 진출이라는 또다른 꿈을 향해 나아간다.',
'이번 구조 조정이 제품을 효과적으로 개발하고 판매하기 위한 회사의 능력 강화 조처임을 이해해 주시리라 생각합니다.',
'요즘 이 프로그램 녹화하며 많은 걸 느낀다'
'배낭 여행은 우리들이 어렸을 때 허용디지 않으면 우울 해 한다'
'그 소녀는 단지 늑대처럼 울부짖을 수 있을 뿐이었다']
